In [1]:
# default_exp exec.train_tacotron2

In [2]:
# export
from uberduck_ml_dev.trainer.tacotron2 import Tacotron2Trainer
from uberduck_ml_dev.vendor.tfcompat.hparam import HParams
from uberduck_ml_dev.trainer.tacotron2 import DEFAULTS as TACOTRON2_TRAINER_DEFAULTS
import argparse
import sys
import json
import torch
from torch import multiprocessing as mp

################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [3]:
# export
def parse_args(args):
    parser = argparse.ArgumentParser()
    parser.add_argument("--config", help="Path to JSON config")
    args = parser.parse_args(args)
    return args


def run(rank, device_count, hparams):
    trainer = Tacotron2Trainer(hparams, rank=rank, world_size=device_count)
    try:
        trainer.train()
    except Exception as e:
        print(f"Exception raised while training: {e}")
        # TODO: save state.
        raise e

In [4]:
from uberduck_ml_dev.models.tacotron2.tacotron2 import (
    NON_ATTENTIVE_DEFAULTS as NON_ATTENTIVE_DEFAULTS,
)

NON_ATTENTIVE_DEFAULTS.decoder_rnn_dim_nlayers

2

In [5]:
# skip
config = NON_ATTENTIVE_DEFAULTS.values()
with open("test/fixtures/ljtest/taco2_lj2lj.json") as f:
    config.update(json.load(f))
config["speaker_embedding_dim"] = 1
config["decoder_rnn_dim_nlayers"] = 2
config["ignore_layers"] = [
    "speaker_embedding.weight",
    "decoder.attention_rnn.weight_ih",
    "decoder.attention_rnn.weight_hh",
    "decoder.attention_rnn.bias_ih",
    "decoder.attention_rnn.bias_hh",
    "decoder.attention_layer.query_layer.linear_layer.weight",
    "decoder.attention_layer.memory_layer.linear_layer.weight",
    "decoder.attention_layer.v.linear_layer.weight",
    "decoder.attention_layer.location_layer.location_conv.conv.weight",
    "decoder.attention_layer.location_layer.location_dense.linear_layer.weight",
    "decoder.decoder_rnn.weight_ih",
    "decoder.decoder_rnn.weight_hh",
    "decoder.decoder_rnn.bias_ih",
    "decoder.decoder_rnn.bias_hh",
    "decoder.linear_projection.linear_layer.weight",
]
hparams = HParams(**config)

print(hparams)
if hparams.distributed_run:
    device_count = torch.cuda.device_count()
    mp.spawn(run, (device_count, hparams), device_count)
else:
    run(None, None, hparams)

[('attention_dim', 128), ('attention_location_kernel_size', 31), ('attention_location_n_filters', 32), ('attention_rnn_dim', 1024), ('batch_size', 16), ('checkpoint_name', None), ('checkpoint_path', 'test/fixtures/results/checkpoints'), ('coarse_n_frames_per_step', None), ('compute_durations', True), ('cudnn_enabled', True), ('dataset_path', './dataset'), ('debug', False), ('decoder_rnn_dim', 1024), ('decoder_rnn_dim_nlayers', 2), ('distributed_run', False), ('duration_lstm_dim', 1024), ('encoder_embedding_dim', 512), ('encoder_kernel_size', 5), ('encoder_n_convolutions', 3), ('epochs', 5), ('epochs_per_checkpoint', 4), ('filter_length', 1024), ('fp16_run', False), ('gate_threshold', 0.5), ('grad_clip_thresh', 1.0), ('gst_type', None), ('has_speaker_embedding', False), ('hop_length', 256), ('ignore_layers', ['speaker_embedding.weight', 'decoder.attention_rnn.weight_ih', 'decoder.attention_rnn.weight_hh', 'decoder.attention_rnn.bias_ih', 'decoder.attention_rnn.bias_hh', 'decoder.attenti

[NeMo W 2022-02-22 06:53:55 features:232] Using torch_stft is deprecated and will be removed in 1.1.0. Please set stft_conv and stft_exact_pad to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-02-22 06:53:55 features:255] PADDING: 1
[NeMo I 2022-02-22 06:53:55 features:265] STFT using conv
[NeMo I 2022-02-22 06:53:59 save_restore_connector:157] Model EncDecCTCModel was successfully restored from /home/s_uberduck_ai/.cache/torch/NeMo/NeMo_1.7.0rc0/qn5x5_libri_tts_phonemes/656c7439dd3a0d614978529371be498b/qn5x5_libri_tts_phonemes.nemo.


[NeMo E 2022-02-22 06:53:59 vocabs:324] Torch distributed needs to be initialized before you initialized <nemo.collections.common.data.vocabs.Phonemes object at 0x7fbc478de730>. This class is prone to data access race conditions. Now downloading corpora from global rank 0. If other ranks pass this before rank 0, errors might result.


[NeMo I 2022-02-22 06:54:00 cloud:56] Found existing object /home/s_uberduck_ai/.cache/torch/NeMo/NeMo_1.7.0rc0/qn5x5_libri_tts_phonemes/656c7439dd3a0d614978529371be498b/qn5x5_libri_tts_phonemes.nemo.
[NeMo I 2022-02-22 06:54:00 cloud:62] Re-using file from: /home/s_uberduck_ai/.cache/torch/NeMo/NeMo_1.7.0rc0/qn5x5_libri_tts_phonemes/656c7439dd3a0d614978529371be498b/qn5x5_libri_tts_phonemes.nemo
[NeMo I 2022-02-22 06:54:00 common:704] Instantiating model from pre-trained checkpoint


[NeMo W 2022-02-22 06:54:00 features:232] Using torch_stft is deprecated and will be removed in 1.1.0. Please set stft_conv and stft_exact_pad to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-02-22 06:54:00 features:255] PADDING: 1
[NeMo I 2022-02-22 06:54:00 features:265] STFT using conv
[NeMo I 2022-02-22 06:54:01 save_restore_connector:157] Model EncDecCTCModel was successfully restored from /home/s_uberduck_ai/.cache/torch/NeMo/NeMo_1.7.0rc0/qn5x5_libri_tts_phonemes/656c7439dd3a0d614978529371be498b/qn5x5_libri_tts_phonemes.nemo.


[NeMo E 2022-02-22 06:54:01 vocabs:324] Torch distributed needs to be initialized before you initialized <nemo.collections.common.data.vocabs.Phonemes object at 0x7fbc3d1742b0>. This class is prone to data access race conditions. Now downloading corpora from global rank 0. If other ranks pass this before rank 0, errors might result.

KeyboardInterrupt



In [6]:
# skip
config = TACOTRON2_TRAINER_DEFAULTS.values()
config['batch_size'] = 4
with open("test/fixtures/ljtest/taco2_lj2lj.json") as f:
    config.update(json.load(f))
hparams = HParams(**config)
print(hparams)
if hparams.distributed_run:
    device_count = torch.cuda.device_count()
    mp.spawn(run, (device_count, hparams), device_count)
else:
    run(None, None, hparams)

[('attention_dim', 128), ('attention_location_kernel_size', 31), ('attention_location_n_filters', 32), ('attention_rnn_dim', 1024), ('batch_size', 16), ('checkpoint_name', None), ('checkpoint_path', 'test/fixtures/results/checkpoints'), ('coarse_n_frames_per_step', None), ('compute_durations', False), ('cudnn_enabled', True), ('dataset_path', './dataset'), ('debug', False), ('decay_rate', 8000), ('decay_start', 15000), ('decoder_rnn_dim', 1024), ('distributed_run', False), ('encoder_embedding_dim', 512), ('encoder_kernel_size', 5), ('encoder_n_convolutions', 3), ('epochs', 5), ('epochs_per_checkpoint', 4), ('filter_length', 1024), ('fp16_run', False), ('gate_threshold', 0.5), ('grad_clip_thresh', 1.0), ('gst_type', None), ('has_speaker_embedding', False), ('hop_length', 256), ('ignore_layers', ['speaker_embedding.weight']), ('include_durations', False), ('include_f0', False), ('learning_rate', 0.001), ('location_specific_attention', True), ('log_dir', 'test/fixtures/results/logs'), ('m

[NeMo W 2022-02-22 06:54:27 features:232] Using torch_stft is deprecated and will be removed in 1.1.0. Please set stft_conv and stft_exact_pad to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-02-22 06:54:27 features:255] PADDING: 1
[NeMo I 2022-02-22 06:54:27 features:265] STFT using conv
[NeMo I 2022-02-22 06:54:28 save_restore_connector:157] Model EncDecCTCModel was successfully restored from /home/s_uberduck_ai/.cache/torch/NeMo/NeMo_1.7.0rc0/qn5x5_libri_tts_phonemes/656c7439dd3a0d614978529371be498b/qn5x5_libri_tts_phonemes.nemo.


[NeMo E 2022-02-22 06:54:28 vocabs:324] Torch distributed needs to be initialized before you initialized <nemo.collections.common.data.vocabs.Phonemes object at 0x7fbc3c1e0b20>. This class is prone to data access race conditions. Now downloading corpora from global rank 0. If other ranks pass this before rank 0, errors might result.


[NeMo I 2022-02-22 06:54:28 cloud:56] Found existing object /home/s_uberduck_ai/.cache/torch/NeMo/NeMo_1.7.0rc0/qn5x5_libri_tts_phonemes/656c7439dd3a0d614978529371be498b/qn5x5_libri_tts_phonemes.nemo.
[NeMo I 2022-02-22 06:54:28 cloud:62] Re-using file from: /home/s_uberduck_ai/.cache/torch/NeMo/NeMo_1.7.0rc0/qn5x5_libri_tts_phonemes/656c7439dd3a0d614978529371be498b/qn5x5_libri_tts_phonemes.nemo
[NeMo I 2022-02-22 06:54:28 common:704] Instantiating model from pre-trained checkpoint


...

[NeMo E 2022-02-22 06:54:29 vocabs:324] Torch distributed needs to be initialized before you initialized <nemo.collections.common.data.vocabs.Phonemes object at 0x7fbc3c021ee0>. This class is prone to data access race conditions. Now downloading corpora from global rank 0. If other ranks pass this before rank 0, errors might result.


Not using any style tokens
Starting warm_start 27720.967232332
Ending warm_start 27721.03247772


[NeMo W 2022-02-22 06:54:31 nemo_logging:349] /opt/conda/envs/uberduck/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
      warnings.warn(warning.format(ret))
    
[NeMo W 2022-02-22 06:54:33 nemo_logging:349] /mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/uberduck_ml_dev/trainer/tacotron2.py:545: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
      grad_norm = torch.nn.utils.clip_grad_norm(
    


epoch: 0/5 | batch: 0/1 | loss: 0.84 | mel: 1.74 | gate: 0.899 | t: 0.40s | w: 0.00h


KeyboardInterrupt: 

In [8]:
# skip
config = TACOTRON2_TRAINER_DEFAULTS.values()
config['batch_size'] = 4
with open("test/fixtures/ljtest/taco2_lj2lj.json") as f:
    config.update(json.load(f))
config['distributed_run'] = True
hparams = HParams(**config)
if hparams.distributed_run:
    device_count = torch.cuda.device_count()
    mp.spawn(run, (device_count, hparams), device_count)
else:
    run(None, None, hparams)

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/conda/envs/uberduck/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/conda/envs/uberduck/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'run' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/conda/envs/uberduck/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/conda/envs/uberduck/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'run' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/conda/envs/uberduck/lib/python3.9/multiprocessing/spawn

ProcessExitedException: process 2 terminated with exit code 1

In [ ]:
# export
try:
    from nbdev.imports import IN_NOTEBOOK
except:
    IN_NOTEBOOK = False
if __name__ == "__main__" and not IN_NOTEBOOK:
    args = parse_args(sys.argv[1:])
    config = TACOTRON2_TRAINER_DEFAULTS.values()
    config['distributed_run'] = True
    if args.config:
        with open(args.config) as f:
            config.update(json.load(f))
    config.update(vars(args))
    hparams = HParams(**config)
    if hparams.distributed_run:
        device_count = torch.cuda.device_count()
        mp.spawn(run, (device_count, hparams), device_count)
    else:
        run(None, None, hparams)

In [4]:
config = TACOTRON2_TRAINER_DEFAULTS.values()
with open("test/fixtures/ljtest/taco2_lj2lj.json") as f:
    config.update(json.load(f))
hparams = HParams(**config)

rank = None
device_count = None
trainer = Tacotron2Trainer(hparams, rank=rank, world_size=device_count)
trainer.train()

TTSTrainer start 23580.806363276
Initializing trainer with hparams:
{'attention_dim': 128,
 'attention_location_kernel_size': 31,
 'attention_location_n_filters': 32,
 'attention_rnn_dim': 1024,
 'batch_size': 16,
 'checkpoint_name': None,
 'checkpoint_path': 'test/fixtures/results/checkpoints',
 'coarse_n_frames_per_step': None,
 'compute_durations': False,
 'cudnn_enabled': True,
 'dataset_path': './dataset',
 'debug': False,
 'decay_rate': 8000,
 'decay_start': 15000,
 'decoder_rnn_dim': 1024,
 'distributed_run': False,
 'encoder_embedding_dim': 512,
 'encoder_kernel_size': 5,
 'encoder_n_convolutions': 3,
 'epochs': 5,
 'epochs_per_checkpoint': 4,
 'filter_length': 1024,
 'fp16_run': False,
 'gate_threshold': 0.5,
 'grad_clip_thresh': 1.0,
 'gst_type': None,
 'has_speaker_embedding': False,
 'hop_length': 256,
 'ignore_layers': ['speaker_embedding.weight'],
 'include_durations': False,
 'include_f0': False,
 'learning_rate': 0.001,
 'location_specific_attention': True,
 'log_dir': 

[NeMo W 2022-02-22 05:45:30 features:232] Using torch_stft is deprecated and will be removed in 1.1.0. Please set stft_conv and stft_exact_pad to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-02-22 05:45:30 features:255] PADDING: 1
[NeMo I 2022-02-22 05:45:30 features:265] STFT using conv
[NeMo I 2022-02-22 05:45:34 save_restore_connector:157] Model EncDecCTCModel was successfully restored from /home/s_uberduck_ai/.cache/torch/NeMo/NeMo_1.7.0rc0/qn5x5_libri_tts_phonemes/656c7439dd3a0d614978529371be498b/qn5x5_libri_tts_phonemes.nemo.


[NeMo E 2022-02-22 05:45:34 vocabs:324] Torch distributed needs to be initialized before you initialized <nemo.collections.common.data.vocabs.Phonemes object at 0x7f5c2b6b9d00>. This class is prone to data access race conditions. Now downloading corpora from global rank 0. If other ranks pass this before rank 0, errors might result.


[NeMo I 2022-02-22 05:45:34 cloud:56] Found existing object /home/s_uberduck_ai/.cache/torch/NeMo/NeMo_1.7.0rc0/qn5x5_libri_tts_phonemes/656c7439dd3a0d614978529371be498b/qn5x5_libri_tts_phonemes.nemo.
[NeMo I 2022-02-22 05:45:34 cloud:62] Re-using file from: /home/s_uberduck_ai/.cache/torch/NeMo/NeMo_1.7.0rc0/qn5x5_libri_tts_phonemes/656c7439dd3a0d614978529371be498b/qn5x5_libri_tts_phonemes.nemo
[NeMo I 2022-02-22 05:45:34 common:704] Instantiating model from pre-trained checkpoint


...

[NeMo I 2022-02-22 05:45:35 features:255] PADDING: 1
[NeMo I 2022-02-22 05:45:35 features:265] STFT using conv
[NeMo I 2022-02-22 05:45:35 save_restore_connector:157] Model EncDecCTCModel was successfully restored from /home/s_uberduck_ai/.cache/torch/NeMo/NeMo_1.7.0rc0/qn5x5_libri_tts_phonemes/656c7439dd3a0d614978529371be498b/qn5x5_libri_tts_phonemes.nemo.


[NeMo E 2022-02-22 05:45:35 vocabs:324] Torch distributed needs to be initialized before you initialized <nemo.collections.common.data.vocabs.Phonemes object at 0x7f5c20f66670>. This class is prone to data access race conditions. Now downloading corpora from global rank 0. If other ranks pass this before rank 0, errors might result.


Not using any style tokens
Starting warm_start 23587.086845543
Ending warm_start 23587.150656177


[NeMo W 2022-02-22 05:45:37 nemo_logging:349] /opt/conda/envs/uberduck/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
      warnings.warn(warning.format(ret))
    
[NeMo W 2022-02-22 05:45:39 nemo_logging:349] /mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/uberduck_ml_dev/trainer/tacotron2.py:545: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
      grad_norm = torch.nn.utils.clip_grad_norm(
    


epoch: 0/5 | batch: 0/1 | loss: 0.84 | mel: 1.74 | gate: 0.899 | t: 0.38s | w: 0.00h
epoch: 1/5 | batch: 0/1 | loss: 4.20 | mel: 5.09 | gate: 0.899 | t: 9.23s | w: 0.01h
epoch: 2/5 | batch: 0/1 | loss: 1.22 | mel: 2.12 | gate: 0.899 | t: 6.43s | w: 0.01h
epoch: 3/5 | batch: 0/1 | loss: 1.48 | mel: 2.38 | gate: 0.899 | t: 6.20s | w: 0.01h
epoch: 4/5 | batch: 0/1 | loss: 1.15 | mel: 2.05 | gate: 0.899 | t: 6.18s | w: 0.01h


In [10]:
rank = None
device_count = None
trainer = Tacotron2Trainer(hparams, rank=rank, world_size=device_count)
trainer.train()

TTSTrainer start 23132.803585774
Initializing trainer with hparams:
{'attention_dim': 128,
 'attention_location_kernel_size': 31,
 'attention_location_n_filters': 32,
 'attention_rnn_dim': 1024,
 'batch_size': 16,
 'checkpoint_name': None,
 'checkpoint_path': 'test/fixtures/results/checkpoints',
 'coarse_n_frames_per_step': None,
 'compute_durations': True,
 'cudnn_enabled': True,
 'dataset_path': './dataset',
 'debug': False,
 'decoder_rnn_dim': 1024,
 'decoder_rnn_dim_nlayers': 2,
 'distributed_run': False,
 'duration_lstm_dim': 1024,
 'encoder_embedding_dim': 512,
 'encoder_kernel_size': 5,
 'encoder_n_convolutions': 3,
 'epochs': 5,
 'epochs_per_checkpoint': 4,
 'filter_length': 1024,
 'fp16_run': False,
 'gate_threshold': 0.5,
 'grad_clip_thresh': 1.0,
 'gst_type': None,
 'has_speaker_embedding': False,
 'hop_length': 256,
 'ignore_layers': ['speaker_embedding.weight',
                   'decoder.attention_rnn.weight_ih',
                   'decoder.attention_rnn.weight_hh',
     

[NeMo W 2022-02-22 05:38:02 features:232] Using torch_stft is deprecated and will be removed in 1.1.0. Please set stft_conv and stft_exact_pad to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-02-22 05:38:02 features:255] PADDING: 1
[NeMo I 2022-02-22 05:38:02 features:265] STFT using conv
[NeMo I 2022-02-22 05:38:03 save_restore_connector:157] Model EncDecCTCModel was successfully restored from /home/s_uberduck_ai/.cache/torch/NeMo/NeMo_1.7.0rc0/qn5x5_libri_tts_phonemes/656c7439dd3a0d614978529371be498b/qn5x5_libri_tts_phonemes.nemo.


[NeMo E 2022-02-22 05:38:03 vocabs:324] Torch distributed needs to be initialized before you initialized <nemo.collections.common.data.vocabs.Phonemes object at 0x7fd9885581c0>. This class is prone to data access race conditions. Now downloading corpora from global rank 0. If other ranks pass this before rank 0, errors might result.


[NeMo I 2022-02-22 05:38:03 cloud:56] Found existing object /home/s_uberduck_ai/.cache/torch/NeMo/NeMo_1.7.0rc0/qn5x5_libri_tts_phonemes/656c7439dd3a0d614978529371be498b/qn5x5_libri_tts_phonemes.nemo.
[NeMo I 2022-02-22 05:38:03 cloud:62] Re-using file from: /home/s_uberduck_ai/.cache/torch/NeMo/NeMo_1.7.0rc0/qn5x5_libri_tts_phonemes/656c7439dd3a0d614978529371be498b/qn5x5_libri_tts_phonemes.nemo
[NeMo I 2022-02-22 05:38:03 common:704] Instantiating model from pre-trained checkpoint


[NeMo W 2022-02-22 05:38:04 features:232] Using torch_stft is deprecated and will be removed in 1.1.0. Please set stft_conv and stft_exact_pad to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2022-02-22 05:38:04 features:255] PADDING: 1
[NeMo I 2022-02-22 05:38:04 features:265] STFT using conv
[NeMo I 2022-02-22 05:38:04 save_restore_connector:157] Model EncDecCTCModel was successfully restored from /home/s_uberduck_ai/.cache/torch/NeMo/NeMo_1.7.0rc0/qn5x5_libri_tts_phonemes/656c7439dd3a0d614978529371be498b/qn5x5_libri_tts_phonemes.nemo.


[NeMo E 2022-02-22 05:38:04 vocabs:324] Torch distributed needs to be initialized before you initialized <nemo.collections.common.data.vocabs.Phonemes object at 0x7fd988316670>. This class is prone to data access race conditions. Now downloading corpora from global rank 0. If other ranks pass this before rank 0, errors might result.


Not using any style tokens
Starting warm_start 23136.399235868
WARNING! Attempting to load a model with out the decoder.range_predictor.lstm.weight_ih_l0 layer. This could lead to unexpected results during evaluation.
WARNING! Attempting to load a model with out the decoder.range_predictor.lstm.weight_hh_l0 layer. This could lead to unexpected results during evaluation.
WARNING! Attempting to load a model with out the decoder.range_predictor.lstm.bias_ih_l0 layer. This could lead to unexpected results during evaluation.
WARNING! Attempting to load a model with out the decoder.range_predictor.lstm.bias_hh_l0 layer. This could lead to unexpected results during evaluation.
WARNING! Attempting to load a model with out the decoder.range_predictor.lstm.weight_ih_l0_reverse layer. This could lead to unexpected results during evaluation.
WARNING! Attempting to load a model with out the decoder.range_predictor.lstm.weight_hh_l0_reverse layer. This could lead to unexpected results during evaluat

epoch: 0/5 | batch: 0/1 | loss: 41.55 | mel: 62.12 | t: 16.05s | w: 0.01h
epoch: 1/5 | batch: 0/1 | loss: 32.81 | mel: 53.09 | t: 40.19s | w: 0.02h
epoch: 2/5 | batch: 0/1 | loss: 14.77 | mel: 35.00 | t: 34.39s | w: 0.03h
epoch: 3/5 | batch: 0/1 | loss: 7.41 | mel: 27.54 | t: 34.51s | w: 0.04h
epoch: 4/5 | batch: 0/1 | loss: 5.20 | mel: 25.32 | t: 34.40s | w: 0.05h
